# FM 24 Moneyball

In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import re

# Import created functions
import sys
sys.path.append(r'C:\Users\kevmi\Documents\FM 24 Moneyball\FM-24-Moneyball')

from money_ball_util import *

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

%matplotlib inline

## Ingest and clean data

In [4]:
### Read raw data

# Select file path for raw data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM 24 exports\Summer 2024\Defensive Midfielders.html"

# Ingest in a dataframe
df_players_raw = pd.read_html(path, encoding='utf-8')[0]

# Format column names 
df_players_raw.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players_raw.columns]

# ---------------------------------------------------------------------------------------------------------------------

### Read processed Data lab data

# Select file path for processed data
path = r"C:\Users\kevmi\Documents\FM 24 Moneyball\FM Data Lab\Man Utd\FM Data Lab outputs\Summer 2024\Defensive Midfielders SL.csv"

# Ingest in a dataframe 
df_players = pd.read_csv(path)   

# Format column names 
df_players.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_players.columns]

# Bring ID from raw data
df_players = df_players.merge(df_players_raw[['name', 'age', 'height', 'weight', 'uid']], 
                 left_on=['name', 'age', 'height', 'weight'], 
                 right_on=['name', 'age', 'height', 'weight']).reset_index().rename(columns={'level_0': 'id'})

df_players['id'] = df_players['uid']

df_players.drop(columns='uid', inplace=True)

## Data cleaning

In [ ]:
df_players.head()

In [5]:
# # Keep only natural DMs
# path = 'data/Natural DMs.html'

# # Ingest in a dataframe
# df_dms = pd.read_html(path, encoding='utf-8')[0]

# # Format column names 
# df_dms.columns = [x.lower().strip().replace(' ', '_').replace('-', '_') for x in df_dms.columns]

# df_dms['natural_dm'] = 'Yes'

# df_dms = df_dms[['uid', 'natural_dm']]

# df_players = df_players.merge(df_dms, 
#                               left_on='id',
#                               right_on='uid',
#                               how='left')

# df_players = df_players[df_players['natural_dm'] == 'Yes']

# Get price estimation values
df_players['price_estimate_M'] = df_players['transfer_value'].apply(parse_transfer_value) / 1000000

## Passing analysis

### Full universe

In [ ]:
# Initialize list of targets
targets = df_players['id']

scatter_plot(df_players,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=True
)

In [ ]:
# Filter data 
mask = (
        # (df_players['passes_attempted/90'] >= 45) & \
        # (df_players['progressive_pass_rate'] >= 7) & \
        # (df_players['pass_completion_%'] >= 85) & \
        # (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='passes_attempted/90', 
             y_metric='progressive_pass_rate',
             z_metric='pass_completion_%',
             title='Passing quality', 
             show_name_label=True
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Possession Analysis

In [ ]:
# Full universe 
scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=True
)

In [ ]:
# Filter data 
mask = (
        (df_players['possession_lost/90'] <= 8) & \
        (df_players['possession_won/90'] >= 7.5) & \
        (df_players['poss+_/90'] >= 0) & \
        (df_players['age'] <= 26) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='possession_lost/90', 
             y_metric='possession_won/90',
             z_metric='poss+_/90',
             title='Possesion quality', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = list(df_targets['id'])

## Aggresion vs Control
Identify aggresive and foul prones targets

In [ ]:
# Full universe 
scatter_plot(df_targets,
             x_metric='tackles_completed/90', 
             y_metric='fouls_committed/90',
             z_metric='tackle_completion_%',
             title='Aggresion - Control', 
             show_name_label=True
)

In [ ]:
# Compute tackes/fouls ratio
df_players['tackles_comp-fouls/90'] = df_players['tackles_completed/90'] / df_players['fouls_committed/90']
# Filter data 
mask = (
        # (df_players['tackles_comp-fouls/90'] <= 2) & \
        # (df_players['tackles_completed/90'] >= 1.5) & \
        # (df_players['tackle_completion_%'] >= 65) & \
        #(df_players['price_estimate_M'] <= 40) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='tackles_completed/90', 
             y_metric='fouls_committed/90',
             z_metric='tackle_completion_%',
             title='Aggresion - Control', 
             show_name_label=False
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Pressure quality

In [ ]:
# Full universe 
scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

In [ ]:
# Filter data 
mask = (
        (df_players['pressures_completed/90'] >= 2.0) & \
        (df_players['pressure_success_%'] >= 20) & \
        (df_players['duels_win_%'] >= 50) & \
        (df_players['id'].isin(targets)) | \
        (df_players['name'].str.contains('Barreiro')) 
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='pressures_completed/90', 
             y_metric='pressure_success_%',
             z_metric='duels_win_%',
             title='Pressure quality', 
             show_name_label=True
)

# Keep only players that fulfill criteria
targets = df_targets['id']

## Presence

In [ ]:
# Full universe 
scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
) 

In [ ]:
df_players[df_players['name'] == 'Casemiro']

In [ ]:
# Filter data 
mask = (
        (df_players['interceptions/90'] >= 1.4) & \
        (df_players['blocks/90'] >= 0.25) & \
        (df_players['clearances/90'] >= 0.5) & \
        (df_players['id'].isin(targets))
        
        
)

df_targets = df_players[mask]

scatter_plot(df_targets,
             x_metric='interceptions/90', 
             y_metric='blocks/90',
             z_metric='clearances/90',
             title='Presence', 
             show_name_label=True
)

#df_targets = df_players[mask]

## Distance

In [ ]:
# Full universe 
scatter_plot(df_targets,
             x_metric='distance_covered/90', 
             y_metric='defensive_actions/90',
             z_metric='creative_actions/90',
             title='Distance', 
             show_name_label=True
) 

In [ ]:
df_targets.groupby('division')['id'].count()

In [ ]:
df_targets.groupby('division').get_group('Sky Bet Championship').sort_values('age')